In [ ]:
import os
import torch

from torch.utils.data import DataLoader, Dataset

import numpy as np
import plotly.graph_objects as go
import plotly.express as px

import sys
sys.path.append('/mnt/raid/C1_ML_Analysis/source/ShapeAXI/src')

import pandas as pd

sys.path.append('/mnt/raid/C1_ML_Analysis/source/diffusion-point-cloud')

from shapeaxi.saxi_dataset import ShapeNetCoreDataModule, SaxiDataModuleVF
from shapeaxi import saxi_nets
from shapeaxi.saxi_nets_lightning import SaxiFlowVAE
from shapeaxi.saxi_layers import *
from shapeaxi.saxi_flows import SaxiCouplingFlow
from shapeaxi.saxi_diffusion import SaxiNoiseScheduler
from shapeaxi.saxi_transforms import TrainTransform
from shapeaxi import utils

from pytorch3d.structures import (
    Meshes,
    Pointclouds,)

from pytorch3d.ops import knn_points, knn_gather, sample_farthest_points

from pytorch3d.vis.plotly_vis import plot_scene

import argparse


In [ ]:
device = torch.device("cuda:0")


In [ ]:
# pc = np.load('/mnt/raid/C1_ML_Analysis/simulated_data_export/skeletons_pc/FAM-025-0447-5_skeleton.npy')

p = np.load('/mnt/raid/C1_ML_Analysis/simulated_data_export/studies_fetus.npy')
p = TrainTransform()(torch.tensor(p))
# p = torch.tensor(p)
pc = Pointclouds(points=p[0:2])

pc_d = {}
for idx in range(len(pc.points_list())):
    pc_d[idx] = {idx: pc[idx]}

fig = plot_scene(
    pc_d,
    ncols=len(pc.points_list())
)
fig.show()

In [ ]:
sys.path.append('/mnt/raid/C1_ML_Analysis/source/HilbertSort/src')
import hilbertsort as hs
from scipy.spatial import distance_matrix

In [ ]:
# Parameters
bins = 32
radius = 1.25
origin = (0,0,0)

# Initiate Sorter
sorter = hs.HilbertSort3D(origin=origin, radius=radius, bins=bins)

p_sorted = sorter.sort(p[0])
p_sorted.shape


In [ ]:
p_rgb = torch.tensor(p_sorted).view(64, 64, 3).detach().cpu().numpy()
p_rgb = p_rgb - p_rgb.min()
p_rgb = p_rgb / p_rgb.max()
px.imshow(p_rgb)

In [ ]:
# class HilbertSort3D:
#     def __init__(self, origin=(0.0, 0.0, 0.0), radius=1.0, bins=32):
#         """
#         Initialize HilbertSort3D.
#         :param origin: Tuple of floats, the origin point for the Hilbert sorting.
#         :param radius: Float, radius of the space.
#         :param bins: Int, number of bins (must be a power of 2).
#         """
#         self.origin = torch.tensor(origin, dtype=torch.float32)
#         self.radius = radius
#         self.bins = bins
#         self.curve = self._generate_hilbert_curve(bins)

#     def _generate_hilbert_curve(self, bins):
#         """
#         Generate the 3D Hilbert curve indices for given bins.
#         Returns a tensor mapping bin (x, y, z) coordinates to their 1D Hilbert order.
#         """
#         size = bins
#         indices = torch.arange(size**3).view(size, size, size)
#         return indices

#     def sort(self, point_cloud):
#         """
#         Sort a batch of point clouds using Hilbert sorting.
#         :param point_cloud: Tensor of shape (B, N, 3), where B is batch size, N is number of points.
#         :return: Sorted tensor of shape (B, N, 3).
#         """
#         B, N, _ = point_cloud.shape

#         # Center and normalize data
#         point_cloud = point_cloud - self.origin
#         bin_interval = (self.radius * 2) / self.bins
#         bins = ((point_cloud / bin_interval) + (self.bins // 2)).long()
#         bins = torch.clamp(bins, 0, self.bins - 1)

#         # Flatten the bin coordinates into a 1D Hilbert index
#         hilbert_indices = self.curve[bins[:, :, 0], bins[:, :, 1], bins[:, :, 2]]

#         # Sort each batch of point clouds by their Hilbert indices
#         sorted_indices = torch.argsort(hilbert_indices, dim=1)
#         sorted_points = torch.gather(point_cloud, 1, sorted_indices.unsqueeze(-1).expand(-1, -1, 3))
#         return sorted_points


sorter_pt = saxi_nets.HilbertSort3D(radius=1.25, bins=64)

p_sorted_pt = sorter_pt(p)[0]


In [ ]:
p_rgb = p_sorted_pt.view(-1, 64, 64, 3).detach().cpu().numpy()
p_rgb = p_rgb - p_rgb.min()
p_rgb = p_rgb / p_rgb.max()
px.imshow(p_rgb, binary_string=True, animation_frame=0)

In [ ]:
p_rgb = p_sorted_pt.view(-1, 64, 64, 3).detach().cpu().numpy()
p_rgb = p_rgb - p_rgb.min()
p_rgb = p_rgb / p_rgb.max()
px.imshow(p_rgb, binary_string=True, animation_frame=0)

In [ ]:

pc = Pointclouds(points=p_sorted_pt[0:4])

pc_d = {}
for idx in range(len(pc.points_list())):
    pc_d[idx] = {idx: pc[idx]}

fig = plot_scene(
    pc_d,
    ncols=len(pc.points_list())
)
fig.show()